In [ ]:
%matplotlib notebook

In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
import pandas
import matplotlib.pyplot as plt
from netCDF4 import Dataset

In [4]:
def hvel(r):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [5]:
def pres(r):
    return pch+dph*np.exp(-(rmaxh/r)**bh)

In [6]:
def hvel2d(l1,l2,vtx,vty):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

In [7]:
path='/DATA/critechuser/BestTrack/'

In [9]:
data=Dataset(path+'Allstorms.ibtracs_all.v03r08.nc')

# ANALYSIS STARTS HERE

In [11]:
hurName='SANDY'

In [31]:
data

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Title: IBTrACS-WMO: NetCDF reformat
    Version: v03r08
    Description: IBTrACS-WMO data reformatted to contain 
all data in one netCDF file. Also an attempt has been made
to have the data appear as trajectories in the CDM
    cdm_datatype: Trajectory
    trajectoryDimension: storm
    Conventions: CF-1.0
    metadata_link: gov.noaa.ncdc:C00834
    dimensions(sizes): storm(12635), time(191), ncharsn(13), ncharnm(57), center(26), ncharcn(10)
    variables(dimensions): |S1 storm_sn(storm,ncharsn), |S1 name(storm,ncharnm), int16 numObs(storm), int16 season(storm), int8 track_type(storm), int8 genesis_basin(storm), int8 num_basins(storm), int8 basin(storm,time), int8 wind_avg_period(center), |S1 source(center,ncharcn), float64 source_time(storm,time), int16 lat_for_mapping(storm,time), int16 lon_for_mapping(storm,time), int8 nature_for_mapping(storm,time), int16 source_lat(storm,time,center

In [67]:
datadic={}
for k in data.variables:
    datadic[k]=data.variables[k][:]

In [68]:
datadic.keys()

[u'lat_for_mapping',
 u'source_lat',
 u'storm_sn',
 u'source_time',
 u'num_basins',
 u'numObs',
 u'season',
 u'track_type',
 u'lon_for_mapping',
 u'source_pres',
 u'genesis_basin',
 u'wind_avg_period',
 u'landfall',
 u'source',
 u'source_wind',
 u'dist2land',
 u'basin',
 u'source_lon',
 u'nature_for_mapping',
 u'name']

In [69]:
datadic['storm_sn']=np.array([''.join(x) for x in datadic['storm_sn']])

In [70]:
datadic['name']=np.array([''.join(x) for x in datadic['name']])

In [96]:
datadic['source']=np.array([''.join(x) for x in datadic['source']])

In [98]:
for k in datadic.keys():
  print k, datadic[k].shape    

lat_for_mapping (12635, 191)
source_lat (12635, 191, 26)
storm_sn (12635,)
source_time (12635, 191)
num_basins (12635,)
numObs (12635,)
season (12635,)
track_type (12635,)
lon_for_mapping (12635, 191)
source_pres (12635, 191, 26)
genesis_basin (12635,)
wind_avg_period (26,)
landfall (12635, 191)
source (26,)
source_wind (12635, 191, 26)
dist2land (12635, 191)
basin (12635, 191)
source_lon (12635, 191, 26)
nature_for_mapping (12635, 191)
name (12635,)


In [104]:
datadic['track_type']

array([0, 0, 2, ..., 0, 0, 0], dtype=int8)

In [91]:
pd={k:datadic[k] for k in ('storm_sn','name')}

In [92]:
df=pandas.DataFrame.from_dict(pd,orient='index')

In [93]:
df

,0,1,2,3,4,5,6,7,8,9,...,12625,12626,12627,12628,12629,12630,12631,12632,12633,12634
storm_sn,1842298N11080,1845336N10074,1848011S09079,1848011S09080,1848011S15057,1848011S16057,1848061S12075,1848062S13069,1848112S07084,1848112S07444,...,2014303N13141,2014309N13088,2014329N08131,2014334N02156,2014355S07099,2014362N07130,2015045S12145,2015046S15150,2015068S14114,2015117S12115
name,NOT NAMED,NOT NAMED,NOT NAMED,XXXX848003,XXXX848002,NOT NAMED,XXXX848001,NOT NAMED,XXXX848004,NOT NAMED,...,NURI,FIVE,SINLAKU,HAGUPIT,KATE,JANGMI,LAM,MARCIA,OLWYN,QUANG


In [27]:
i=names[1]

In [ ]:
bul=10

In [ ]:
infolder=sdir+'/{}'.format(bul)

In [ ]:
infofile=infolder+'/info.xml'
info=bxml(infofile)
print info.setexp

In [ ]:
outfile=infolder+'/outData.txt'

In [ ]:
dat=pandas.read_csv(outfile,header=0,delimiter='\t',usecols=np.arange(0,14))
dat

In [ ]:
b=dat['b'].values
vmax=dat['vmax'].values
k=dat['k'].values
rmax=dat['rmax'].values
deltap=dat['deltap'].values
vtrx=dat['vtrx'].values
vtry=dat['vtry'].values

In [ ]:
# choose time
it=0

In [ ]:
bh=b[it]
vmaxh=vmax[it]
kh=k[it]
rmaxh=rmax[it]
dph=deltap[it]
pch=pn-dph
vtx=vtrx[it]
vty=vtry[it]

In [ ]:
rh=np.linspace(0.,5*rmaxh,100)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,hvel(rh))
plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')


plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,pres(rh)/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure [PA]',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:
ni,nj=1000,1000
xl=np.linspace(-maxR, maxR,ni+1)
yl=np.linspace(-maxR, maxR,nj+1)
q1,q2=np.meshgrid(xl,yl)

In [ ]:
zx,zy, pr=hvel2d(q1,q2,vtx,vty)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,pr/100,20)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Pressure contours')


plt.colorbar(label='PA')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,pr[500,:]/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:

vel=np.sqrt(zx**2+zy**2)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,vel,20)
plt.streamplot(q1/1000,q2/1000,zx,zy, color=vel, linewidth=0.1*vel)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Velocity vs radius')

plt.colorbar(label='m/s')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,vel[ni/2,:],label='horizontal')
plt.plot(q2[:,0]/1000,vel[:,nj/2],label='verical')

plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')


plt.legend(loc=0)
plt.show()

In [ ]:
np.savetxt('tmp/phol',[q1[0,:],pr[500,:]])
np.savetxt('tmp/uhol',[q1[0,:],vel[ni/2,:],vel[:,nj/2]])

In [ ]:
pandas.DataFrame({'Min Pressure':[pch/100],'Max Velocity':[vmaxh]})